<img src="https://relevance.ai/wp-content/uploads/2021/11/logo.79f303e-1.svg" width="150" alt="Relevance AI" />
<h5> Developer-first vector platform for ML teams </h5>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RelevanceAI/workflows/blob/main/workflows/cluster/Cluster_Your_Data_with_Relevance_AI_form.ipynb)


In [ ]:
#@title After filling this form, press the top left button.
# You can grab your token here https://cloud.relevance.ai/sdk/api

token = "<copy paste from https://cloud.relevance.ai/sdk/api>"  #@param  {type:"string"}
dataset_id = "<your dataset ID here>"  #@param {type:"string"}
n_clusters = 10  #@param {type: "integer"}
vector_field =  "<your vector field  here>"#@param {type:"string"}
clusteringType =  "community-detection"  #@param {type:"string"}

#@markdown Optional Configurations
cutoff = 0.75 #@param {type:"number"}

project = token.split(':')[0]
api_key = token.split(':')[1]
region = token.split(':')[2]

config = {
  "dataset_id": dataset_id,
  "n_clusters": 10,
  "vector_fields": [vector_field],
  "cutoff": cutoff,
  "clusteringType": clusteringType,
  "region": region,
  "project": project,
  "api_key": api_key,
  "authorizationToken": token
}

import base64
import json

show_warnings_in_logs = False #@param {type:"boolean"}
#@markdown Once the form is filled and you've clicked run, monitor below for logs of it running

# print(json.dumps(config, indent=2))

print("Installing RelevanceAI")

!pip install -q RelevanceAI==2.1.8
## Instantiate client ###
from relevanceai import Client 
client = Client(token=token)

## Checking valid vector field ###
for v in config['vector_fields']:
  if not '_vector_'in v:
    raise ValueError(f"'{v}' is not a valid vector field")


df = client.Dataset(config['dataset_id'])
cluster_method = "kmeans" # setting to default
try:
  if config['clusteringType'] == 'community-detection':
    cluster_method  = "community_detection"
    !pip install -q sentence-transformers==2.2.0
    df.cluster(
        cluster_method,
        cluster_config={"threshold": config['cutoff']},
        vector_fields=config['vector_fields']
    )
  elif config['clusteringType'] == 'kmeans':
    if df.shape[0] < 3000:
      cluster_method = 'kmeans'
      df.cluster(
          config['clusteringType'],
          cluster_config={"n_clusters": config['n_clusters']},
          vector_fields=config['vector_fields']
      )
    else:
      cluster_method = 'minibatchkmeans'
      df.cluster(
          cluster_method,
          cluster_config={"n_clusters": config['n_clusters']},
          vector_fields=config['vector_fields']
      )
  else:
    cluster_method = "kmeans"
    df.cluster(
        cluster_method,
        cluster_config={"n_clusters": config['n_clusters']},
        vector_fields=config['vector_fields']
    )
except Exception as e:
  raise ValueError('Incorrect token provided')

if cluster_method == "community-detection":
    print(f"Finished subclustering your data with {cluster_method} with cutoff={config['cutoff']}, you may close this window.")
elif cluster_method == "kmeans":
    print(f"Finished subclustering your data with {cluster_method} with n_clusters={config['n_clusters']}, you may close this window.")
